In [1]:
from keras.applications.inception_v3 import InceptionV3

Using TensorFlow backend.


In [2]:
pre_trained_model = InceptionV3(input_shape = (150, 150, 3), # Shape of our images
                                include_top = False, # Leave out the last fully connected layer
                                weights = 'imagenet')

87916544/87910968 [==============================] - 4s 0us/step


In [3]:
for layer in pre_trained_model.layers:
  layer.trainable = False

In [4]:
import tensorflow as tf
class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('acc')>0.959):
      print("\nReached 99.9% accuracy so cancelling training!")
      self.model.stop_training = True

In [5]:
from keras.optimizers import RMSprop
from keras.preprocessing.image import ImageDataGenerator
from keras import layers
from keras import Model

from keras.optimizers import RMSprop

# Flatten the output layer to 1 dimension
x = layers.Flatten()(pre_trained_model.output)
# Add a fully connected layer with 1,024 hidden units and ReLU activation
x = layers.Dense(1024, activation='relu')(x)
# Add a dropout rate of 0.2
x = layers.Dropout(0.2)(x)                  
# Add a final sigmoid layer for classification
x = layers.Dense  (1, activation='sigmoid')(x)           

model = Model( pre_trained_model.input, x) 

model.compile(optimizer = RMSprop(lr=0.0001), 
              loss = 'binary_crossentropy', 
              metrics = ['acc'])

In [12]:
train_datagen = ImageDataGenerator(rescale = 1./255.,
                                   rotation_range = 40,
                                   width_shift_range = 0.2,
                                   height_shift_range = 0.2,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

# Note that the validation data should not be augmented!
test_datagen = ImageDataGenerator( rescale = 1.0/255. )

training_set = train_datagen.flow_from_directory('/floyd/input/umich/training',
                                                 target_size = (150, 150),
                                                 batch_size = 16,
                                                 class_mode = 'binary')

test_set = test_datagen.flow_from_directory('/floyd/input/umich/validation',
                                            target_size = (150, 150),
                                            batch_size = 16,
                                            class_mode = 'binary')

Found 105 images belonging to 2 classes.
Found 26 images belonging to 2 classes.


In [13]:
callbacks = myCallback()
model.fit_generator(training_set,
                         steps_per_epoch = 105,
                         epochs = 30,
                         validation_data = test_set,
                         validation_steps = 26,
                         callbacks = [callbacks])


Epoch 1/30
105/105 [==============================] - 265s 3s/step - loss: 0.7753 - acc: 0.7567 - val_loss: 0.5252 - val_acc: 0.7692
Epoch 2/30
105/105 [==============================] - 258s 2s/step - loss: 0.5105 - acc: 0.8101 - val_loss: 0.6223 - val_acc: 0.7692
Epoch 3/30
105/105 [==============================] - 258s 2s/step - loss: 0.4280 - acc: 0.8513 - val_loss: 0.5348 - val_acc: 0.7692
Epoch 4/30
105/105 [==============================] - 257s 2s/step - loss: 0.4526 - acc: 0.8423 - val_loss: 0.5570 - val_acc: 0.7692
Epoch 5/30
105/105 [==============================] - 256s 2s/step - loss: 0.3967 - acc: 0.8570 - val_loss: 0.7605 - val_acc: 0.7692
Epoch 6/30
105/105 [==============================] - 256s 2s/step - loss: 0.4372 - acc: 0.8543 - val_loss: 0.6547 - val_acc: 0.7692
Epoch 7/30
105/105 [==============================] - 257s 2s/step - loss: 0.3864 - acc: 0.8744 - val_loss: 0.6640 - val_acc: 0.7692
Epoch 8/30
105/105 [==============================] - 257s 2s/step - 

In [14]:
model.save('inception_heart_sounds.h5')

In [5]:
from keras.models import load_model
model = load_model('inception_heart_sounds.h5')

In [41]:
from keras.preprocessing import image
import numpy as np

tc = {0:'Murmur', 1: 'Normal'}

test_image = image.load_img('/floyd/input/umich/training/Murmur/17.png', target_size = (150, 150))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)

result  = model.predict(test_image)
print(result)
res=1
if result<0.5:
    res=0
# res = np.argmax(result)
print(tc[res])

[[0.00056984]]
Murmur
